In [20]:
import pandas as pd
import nameparser
import json
import requests
import numpy as np

pro_publica_api_key = "PguHImFJakafbsBpZ2ed21A1HIgvXKiM7tg0BqI0 "
headers = {'X-API-Key': pro_publica_api_key}

In [39]:
committee_assignments = pd.read_csv("../prod_data/committee_assignments_of_interest.csv")


unique_rows_last = committee_assignments.drop_duplicates(subset=['congress', 'chamber', 'committee_id'])[['congress', 'chamber', 'committee_id']]
print(len(committee_assignments['committee_id'].unique()))
unique_rows_last


52


,congress,chamber,committee_id
0,116,house,HSAG
1,116,house,HSSY
2,116,senate,SSAF
11,116,senate,SSAP
31,116,senate,SSAS
...,...,...,...
1487,118,house,HSIG
1494,118,house,HSZS
1503,118,house,HSVC
1507,118,house,HSFD


In [45]:
old_hearings = pd.read_csv("../prod_data/hearings.csv")[['hearing_congress', 'hearing_chamber', 'hearing_committee_id']]

unique_rows_last_old = old_hearings.drop_duplicates(subset=['hearing_congress', 'hearing_chamber', 'hearing_committee_id'])[['hearing_congress', 'hearing_chamber', 'hearing_committee_id']]

unique_rows_last_old.rename(columns={'hearing_congress': 'congress', 'hearing_chamber': 'chamber', 'hearing_committee_id': 'committee_id'}, inplace=True)

unique_rows_last_old

,congress,chamber,committee_id
0,116,house,HSHA
2,116,house,HSAG
49,116,house,HSAP
109,116,house,HSAS
169,116,house,HSED
229,116,house,HSIF
289,116,house,HSBA
295,116,house,HSFA
299,116,house,HSHM
302,116,house,HSII


In [44]:
merged_df = unique_rows_last.merge(unique_rows_last_old, on=['congress', 'chamber', 'committee_id'], how='outer', indicator=True)

# Filter out rows that are found in both DataFrames
unique_to_df1 = merged_df[merged_df['_merge'] == 'left_only'].drop(columns=['_merge'])

unique_to_df1

,congress,chamber,committee_id
2,116,senate,SSAF
3,116,senate,SSAP
4,116,senate,SSAS
5,116,senate,SSBK
6,116,senate,SSCM
...,...,...,...
143,118,house,HSIG
144,118,house,HSZS
145,118,house,HSVC
146,118,house,HSFD


In [29]:
len(old_hearings['hearing_committee_id'].unique())

26

,congress,chamber,committee_id
0,116,house,HSAG
1,116,house,HSSY
2,116,senate,SSAF
11,116,senate,SSAP
31,116,senate,SSAS
...,...,...,...
1487,118,house,HSIG
1494,118,house,HSZS
1503,118,house,HSVC
1507,118,house,HSFD


In [46]:
hearings_dfs = []
for index, row in unique_to_df1.iterrows():
    hearing_congress_list = []
    hearing_chamber_list = []
    hearing_committee_id_list = []
    hearing_date_list = []
    hearing_description_list = []

    offset_value = 0
    congress = row['congress']
    chamber = row['chamber']
    committee_id = row['committee_id']
    url = f'https://api.propublica.org/congress/v1/{congress}/{chamber}/committees/{committee_id}/hearings.json'
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    results_check = response.json()

    if 'results' in response.json().keys():
        results = response.json()['results']
        bill_number = results[0]['num_results']
        times = bill_number // 20 + 1
        while times >=0:
            url = f'https://api.propublica.org/congress/v1/{congress}/{chamber}/committees/{committee_id}/hearings.json?offset={offset_value}'
            response = requests.get(url, headers=headers)
            hearings = response.json()['results'][0]['hearings']

            for hearing in hearings:
                hearing_congress_list.append(congress)
                hearing_chamber_list.append(chamber)
                hearing_committee_id_list.append(committee_id)
                hearing_date_list.append(hearing['date'])
                hearing_description_list.append(hearing['description'])
            offset_value+= 20
            times -= 1
    else:
        pass
    hearings_df = pd.DataFrame({'hearing_congress': hearing_congress_list,
                                    'hearing_chamber': hearing_chamber_list,
                                    'hearing_committee_id': hearing_committee_id_list,
                                    'hearing_date': hearing_date_list,
                                    'hearing_description': hearing_description_list})
    hearings_dfs.append(hearings_df)

    

hearings_df = pd.concat(hearings_dfs)

hearings_df

,hearing_congress,hearing_chamber,hearing_committee_id,hearing_date,hearing_description
0,116.0,senate,SSAF,2020-12-02,Hearings to examine agricultural research and ...
1,116.0,senate,SSAF,2020-06-24,Business meeting to consider original legislat...
2,116.0,senate,SSAF,2020-03-12,Hearings to examine agriculture innovation and...
3,116.0,senate,SSAF,2019-11-07,Hearings to examine implementation of the 2018...
4,116.0,senate,SSAF,2019-10-17,Hearings to examine implementing the 2018 Farm...
...,...,...,...,...,...
1,118.0,senate,SSFR,2023-04-26,Hearings to examine U.S. policy on Tunisia.
2,118.0,senate,SSFR,2023-04-26,Hearings to examine the President&#39;s propos...
0,118.0,senate,SSSB,2023-04-26,An oversight hearing to examine the SBA&#39;s ...
0,118.0,senate,SSVA,2023-04-26,Hearings to examine pending calendar business.


In [51]:
old_hearings = pd.read_csv("../prod_data/hearings.csv")
old_hearings

final_hearings_df = pd.concat([old_hearings, hearings_df])

final_hearings_df.to_csv('../prod_data/committee_hearings.csv', index=False)

In [16]:
hearings_df['hearing_congress'] = hearings_df['hearing_congress'].astype(int)

hearings_df.to_csv('../prod_data/committee_hearings.csv', index=False)